In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report

In [16]:
################ Bon Code ###############
#### charger et lire un dataset [.zip] sur coolab ####

import zipfile
import os
import pandas as pd

import shutil

# # Chemins vers les fichiers zip quand je suis sur google Coolab

chemin_zip = "Bell_DNS Dataset.zip"


# Fonction pour extraire les fichiers zip
def extraire_zip(chemin_zip):
    with zipfile.ZipFile(chemin_zip, 'r') as zip_ref:
        zip_ref.extractall("extraction_temp")  # Extraire les fichiers zip dans un répertoire temporaire

# Fonction pour charger les fichiers CSV d'un type spécifique (stateful ou stateless)
def charger_concatener_donnees(sous_dossier, prefixe):
    # Lister tous les fichiers CSV dans le sous-dossier
    fichiers_csv = [f for f in os.listdir(f"extraction_temp/{sous_dossier}") if f.startswith(prefixe) and f.endswith('.csv')]
    # Lire chaque fichier CSV et le stocker dans une liste de DataFrames
    dataframes = [pd.read_csv(f"extraction_temp/{sous_dossier}/{f}") for f in fichiers_csv]
    # Concaténer les DataFrames en un seul
    return pd.concat(dataframes, ignore_index=True)

# Extraire les fichiers zip
extraire_zip(chemin_zip)



benin_data = charger_concatener_donnees("Bell_DNS Dataset", "features_domain_benign")
malware_data = charger_concatener_donnees("Bell_DNS Dataset", "features-domain_Malware")

phishing_data = charger_concatener_donnees("Bell_DNS Dataset", "features-domain_phishing")
spam_data = charger_concatener_donnees("Bell_DNS Dataset", "features-domain_Spam")

# Supprimer le répertoire temporaire après avoir terminé

# Vérifier si le répertoire temporaire existe
if os.path.exists("extraction_temp"):
    # Supprimer le répertoire temporaire et son contenu
    shutil.rmtree("extraction_temp")


# Maintenant, vous avez vos données prêtes à être utilisées


In [17]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

# Ajout de la colonne 'Class' pour chaque classe
spam_data['class'] = 'Malicioux'
malware_data['class'] = 'Malicioux'
phishing_data['class'] = 'Malicioux'
benin_data['class'] = 'Benin'


# Concaténation des données
all_data = pd.concat([benin_data, spam_data, malware_data, phishing_data])

# Diviser les données en caractéristiques (X) et les étiquettes de classe (y)
X = all_data.drop(columns=['class'])
y = all_data['class']

# Instancier le sous-échantillonneur aléatoire
rus = RandomUnderSampler(sampling_strategy={'Benin': 23716, 'Malicioux': 22929})

# Appliquer le sous-échantillonneur
X_resampled, y_resampled = rus.fit_resample(X, y)

# Recréer un DataFrame avec les données équilibrées
balanced_data = pd.DataFrame(X_resampled, columns=X.columns)
balanced_data['class'] = y_resampled

# Vérifier la taille des classes après équilibrage
print("Taille de la classe Benin après équilibrage :", balanced_data[balanced_data['class'] == 'Benin'].shape[0])
print("Taille de la classe Malicioux après équilibrage :", balanced_data[balanced_data['class'] == 'Malicioux'].shape[0])

# Taille de l'ensemble de données final
print("\n")
print("Taille de l'ensemble de données final :", balanced_data.shape)

print("Nbre de classe:", balanced_data['class'].value_counts())


Taille de la classe Benin après équilibrage : 23716
Taille de la classe Malicioux après équilibrage : 22929


Taille de l'ensemble de données final : (46645, 36)
Nbre de classe: class
Benin        23716
Malicioux    22929
Name: count, dtype: int64


In [18]:
X_numerical_balanced = balanced_data[['Page_Rank','entropy','len','numeric_percentage','dec_32','oc_8','oc_32']]
X_categorical_balanced = balanced_data[['char_distribution','Registrant_Name','Domain_Name',
                        'distance_from_bad_words','Creation_Date_Time','sld','1gram','2gram','3gram',
                        'shortened','obfuscate_at_sign','Country','Organization','State','Emails','Domain_Age']]
y_balanced = balanced_data['class']



print(X_numerical_balanced.shape)
print(X_categorical_balanced.shape)
print(y_balanced.info())

(46645, 7)
(46645, 16)
<class 'pandas.core.series.Series'>
Index: 46645 entries, 20966 to 596
Series name: class
Non-Null Count  Dtype 
--------------  ----- 
46645 non-null  object
dtypes: object(1)
memory usage: 728.8+ KB
None


In [19]:
########### CONTINUER L'EXECUTION ICI ###########

############ je continue ici ###################
from sklearn.impute import SimpleImputer

# Imputer les valeurs manquantes pour les caractéristiques numériques

# Imputer les valeurs manquantes pour les caractéristiques numériques
numerical_imputer = SimpleImputer(strategy='mean')
X_numerical_balanced_imputed = pd.DataFrame(numerical_imputer.fit_transform(X_numerical_balanced), columns=X_numerical_balanced.columns)

from sklearn.impute import SimpleImputer

# Imputer les valeurs manquantes pour les caractéristiques catégorielles

categorical_imputer = SimpleImputer(strategy='most_frequent')
categorical_balanced_imputed = pd.DataFrame(categorical_imputer.fit_transform(X_categorical_balanced),columns=X_categorical_balanced.columns)


In [20]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

#prepocessing des features numeriques soit  avec le  LabelEncoder soit le MinMaxScaler()

# Sélection des fonctionnalités numériques

# numeric_features = X_numerical_imputed[numeric_imputed_list] ## cas ou je veux selectionné certains features

# Création d'un scaler
scaler = MinMaxScaler()

# Ajustement du scaler aux données
scaler.fit(X_numerical_balanced_imputed)

# Transformation des fonctionnalités numériques
scaled_numeric_balanced = scaler.transform(X_numerical_balanced_imputed)

# Apres transformation Création d' un DataFrame à partir des valeurs transformées

scaled_df_balanced = pd.DataFrame(scaled_numeric_balanced, columns=X_numerical_balanced_imputed.columns)


total_size_balanced = scaled_df_balanced.shape

print("Taille totale des caracteristiques numeriques apres preprocessing :", total_size_balanced)


Taille totale des caracteristiques numeriques apres preprocessing : (46645, 7)


In [28]:
from sklearn.preprocessing import OneHotEncoder

# Convertir toutes les valeurs des colonnes mixtes en chaînes de caractères
mixed_columns = ['char_distribution', 'Registrant_Name', 'Domain_Name', 'distance_from_bad_words',
                 'Creation_Date_Time', 'sld', '1gram', '2gram', '3gram', 'Country', 'Organization',
                 'State', 'Emails', 'Domain_Age', 'shortened', 'obfuscate_at_sign']

for col in mixed_columns:
    categorical_balanced_imputed[col] = categorical_balanced_imputed[col].astype(str)

# Afficher les types de données de chaque colonne pour vérifier
for col in categorical_balanced_imputed.columns:
    print(f"{col}: {categorical_balanced_imputed[col].apply(type).unique()}")

# Réencoder les caractéristiques catégoriques
encoder = OneHotEncoder()
X_categorical_encoded = encoder.fit_transform(categorical_balanced_imputed)

# Convertir en DataFrame
X_categorical_encoded_df = pd.DataFrame(X_categorical_encoded.toarray(), columns=encoder.get_feature_names_out(categorical_balanced_imputed.columns))

print(X_categorical_encoded_df)


char_distribution: [<class 'str'>]
Registrant_Name: [<class 'str'>]
Domain_Name: [<class 'str'>]
distance_from_bad_words: [<class 'str'>]
Creation_Date_Time: [<class 'str'>]
sld: [<class 'str'>]
1gram: [<class 'str'>]
2gram: [<class 'str'>]
3gram: [<class 'str'>]
shortened: [<class 'str'>]
obfuscate_at_sign: [<class 'str'>]
Country: [<class 'str'>]
Organization: [<class 'str'>]
State: [<class 'str'>]
Emails: [<class 'str'>]
Domain_Age: [<class 'str'>]


MemoryError: Unable to allocate 61.1 GiB for an array with shape (46645, 175920) and data type float64

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from sklearn.preprocessing import LabelEncoder

# Encoder les étiquettes de classe
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_balanced)

# Concaténer les données numériques et catégorielles prétraitées
X_combined = pd.concat([X_numerical_balanced_imputed, X_categorical_encoded_df], axis=1)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_encoded, test_size=0.2, random_state=42)

# Définir les hyperparamètres à rechercher
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Créer un objet GridSearchCV pour la recherche d'hyperparamètres
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')

# Effectuer la recherche d'hyperparamètres
grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres :")
print(grid_search.best_params_)

# Entraîner le modèle RandomForestClassifier avec les meilleurs hyperparamètres
best_rf_classifier = grid_search.best_estimator_
best_rf_classifier.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = best_rf_classifier.predict(X_test)

# Convertir les prédictions décodées
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Évaluer le modèle
print("\nRapport de classification :")
print(classification_report(y_test, y_pred_decoded))
print("\nMatrice de confusion :")
print(confusion_matrix(y_test, y_pred_decoded))


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Prédire les probabilités des classes positives
y_prob = best_rf_classifier.predict_proba(X_test)[:, 1]

# Calculer les valeurs de FPR (False Positive Rate) et TPR (True Positive Rate)
fpr, tpr, _ = roc_curve(y_test, y_prob)

# Calculer l'AUC (Area Under the Curve)
roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Courbe ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Taux de vrais positifs (TPR)')
plt.title('Courbe ROC')
plt.legend(loc="lower right")
plt.show()

# Matrice de confusion
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_decoded)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Matrice de confusion')
plt.colorbar()
plt.xticks([0, 1], ['Classe 0', 'Classe 1'])
plt.yticks([0, 1], ['Classe 0', 'Classe 1'])
plt.xlabel('Prédictions')
plt.ylabel('Vraies valeurs')
for i in range(2):
    for j in range(2):
        plt.text(j, i, str(cm[i, j]), horizontalalignment="center", color="white" if cm[i, j] > (cm.max() / 2) else "black")
plt.tight_layout()
plt.show()
